In [79]:
import csv
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

In [6]:
df = pd.read_csv('../train_data/bizfeatures_meanfeature.csv')
label_df = pd.read_csv('train.csv')

In [8]:
df = df.sort_values(by='business_id')
df

,business_id,feature
1040,3,"[0.15115895867347717, 0.0732259601354599, 0.11..."
1456,4,"[0.1477070301771164, 0.06797164678573608, 0.13..."
342,5,"[0.1602800190448761, 0.08416581153869629, 0.15..."
347,6,"[0.12783241271972656, 0.09045923501253128, 0.1..."
1441,7,"[0.15980949997901917, 0.07771550118923187, 0.1..."
154,8,"[0.167060986161232, 0.05579552799463272, 0.133..."
799,9,"[0.16387687623500824, 0.08138040453195572, 0.1..."
1360,12,"[0.12542682886123657, 0.0736740306019783, 0.13..."
781,13,"[0.1405954211950302, 0.0837753415107727, 0.115..."
1012,14,"[0.16205096244812012, 0.07431483268737793, 0.1..."


In [91]:
df = df.sort_values(by='business_id')
label_df = label_df.sort_values(by='business_id')

In [132]:
#randomly breaking the dataset into 80:20 ratio for training and testing
msk = np.random.rand(len(df)) < 0.8
train_new_model = df[msk]
test_new_model = df[~msk]
label_train_new_model = label_df[msk]
label_test_new_model = label_df[~msk]

In [133]:
def str_to_nplist(string):
    string = string.replace("[", "")
    string = string.replace("]", "")
    string = string.split(',')
    return np.asarray(string).astype(np.float32)

In [134]:
mean_features = np.asarray([str_to_nplist(i) for i in train_new_model['feature']])
labels = label_train_new_model['labels'].values

In [135]:
mean_features_test = np.asarray([str_to_nplist(i) for i in test_new_model['feature']])
labels_test = label_test_new_model['labels'].values

In [136]:
X = mean_features
Y = labels

In [137]:
label_binarizer = MultiLabelBinarizer()

In [138]:
def convertlabelsfromString(s, ind):
    if type(s) == str:
        s = s.split(" ")
        s = [int(x) for x in s]
        return s
    else:
        return []

In [139]:
Y = [convertlabelsfromString(Y[i],i) for i in range(Y.shape[0])]

In [140]:
Y = label_binarizer.fit_transform(Y)

In [141]:
X.shape

(1579, 2048)

In [142]:
classifier = OneVsRestClassifier(svm.SVC(kernel='linear'))
classifier.fit(X, Y)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1)

In [143]:
asd = classifier.predict(mean_features_test)
qwe = label_binarizer.inverse_transform(asd)

In [144]:
asd.shape

(421, 9)

In [145]:
qwe = label_binarizer.fit_transform(qwe)

In [146]:
Y1 = [convertlabelsfromString(labels_test[i],i) for i in range(labels_test.shape[0])]

In [147]:
Y1 = label_binarizer.fit_transform(Y1)

In [150]:
f1_score(Y1,qwe,average='macro')

0.70242622000345856